In [ ]:
import os, re, csv
import pandas as pd
import numpy as np
import time, re

import nltk
from nltk.corpus import stopwords

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.feature_extraction import text


In [ ]:
pd.read_csv(''~/ccBox/Research_Project/NLP/Master_Files/M006_all_pts_by_day.csv')

In [ ]:
df=pd.read_csv('tfidfscores_per_day_per_patient_no_dcsum_5.5.csv')

In [ ]:
#df65=df[df.AGE>=65]
#df65.to_csv('~/ccBox/Research_Project/NLP/Master_Files/tfidfscores_per_day_per_65patient_4.20.csv')

In [ ]:
df.columns

In [ ]:
df['percent_neg_words']=round(df.total_neg_words/(df.total_neg_words+df.total_pos_words), 2)
df['percent_pos_words']=round(df.total_pos_words/(df.total_neg_words+df.total_pos_words), 2)

In [ ]:
print('Number of unique patients:', len(df.SUBJECT_ID.unique().tolist()))

In [ ]:
#For HD

In [ ]:
df['has_HD_start']=df.dialysis_start_date.map(lambda x: 0 if str(x)=='nan' else 1)
df['num_dialysis_days']=df['num_dialysis_days'].fillna(0)
df['HD_documented']=np.where((df.has_HD_start==1) | (df.num_dialysis_days>0), 1, 0)
df['new_HD']=(df.days_until_dialysis.map(lambda x: int(''.join(re.findall(r'\d', str(x)[0:6]))) if 'days' in str(x) else 0)>=3)


In [ ]:
print('Number of patients on CRRT/HD:', len(df[df.has_HD_start==True].SUBJECT_ID.unique()))
print('Confirmation Number of patients on CRRT/HD:', len(df[df.HD_documented==True].SUBJECT_ID.unique()))
print('New CRRT/HD:', len(df[df.new_HD==True].SUBJECT_ID.unique()))
print('Confirmation HD not on day of admission:', len(df[df.admit_date<df.dialysis_start_date].SUBJECT_ID.unique()))

print('')
HD_df=df[df.has_HD_start==True]
print('HD_df shape', HD_df.shape)
print('HD_df, new HD:', len(HD_df[HD_df.new_HD==True].SUBJECT_ID.unique()))

In [ ]:
#For Trach

In [ ]:
df['has_trach_start']=df.trach_start_date.map(lambda x: 0 if str(x)=='nan' else 1)
df['num_trach_days']=df['num_trach_days'].fillna(0)
df['trach_documented']=np.where((df.has_trach_start==1) | (df.num_trach_days>0), 1, 0)
df['new_trach']=(df.days_until_trach.map(lambda x: int(''.join(re.findall(r'\d', str(x)[0:6]))) if 'days' in str(x) else 0)>=1)

In [ ]:
print('Number of patients with Trach:', len(df[df.has_trach_start==True].SUBJECT_ID.unique()))
print('Confirmation Number of patients with Trach:', len(df[df.trach_documented==True].SUBJECT_ID.unique()))
print('New Trach:', len(df[df.new_trach==True].SUBJECT_ID.unique()))
print('Confirmation New Trach:', len(df[df.admit_date<df.trach_start_date].SUBJECT_ID.unique()))


print('')
trach_df=df[df.has_trach_start==True]
print('trach_df shape', trach_df.shape)
print('trach_df, new trach:', len(trach_df[trach_df.new_trach==True].SUBJECT_ID.unique()))

In [ ]:
#For Pressors

In [ ]:
df['has_pressors_start']=df.pressors_start_date.map(lambda x: 0 if str(x)=='nan' else 1)
df['num_pressor_days']=df['num_pressor_days'].fillna(0)
df['pressors_documented']=np.where((df.has_pressors_start==1) | (df.num_pressor_days>0), 1, 0)
df['new_pressors']=(df.days_until_pressors.map(lambda x: int(''.join(re.findall(r'\d', str(x)[0:6]))) if 'days' in str(x) else 0)>=1)

In [ ]:
print('Number of patients on pressors:', len(df[df.has_pressors_start==True].SUBJECT_ID.unique()))
print('Confirmation Number of patients on pressors:', len(df[df.pressors_documented==True].SUBJECT_ID.unique()))
print('New Pressors started in ICU:', len(df[df.new_pressors==True].SUBJECT_ID.unique()))
print('Confirmation New Pressors:', len(df[df.admit_date<df.pressors_start_date].SUBJECT_ID.unique()))
print('')
pressors_df=df[df.has_pressors_start==True]
print('pressors_df shape', pressors_df.shape)
print('pressors_df, new pressors:', len(pressors_df[pressors_df.new_pressors==True].SUBJECT_ID.unique()))

In [ ]:
#For MV

In [ ]:
df['has_mv_start']=df.mv_start_date.map(lambda x: 0 if str(x)=='nan' else 1)
df['num_mv_days']=df['num_mv_days'].fillna(0)
df['mv_documented']=np.where((df.has_mv_start==1) | (df.num_mv_days>0), 1, 0)
df['new_mv']=(df.days_until_mechvent.map(lambda x: int(''.join(re.findall(r'\d', str(x)[0:6]))) if 'days' in str(x) else 0)>0)

In [ ]:
print('Number of patients on MechVent:', len(df[df.has_mv_start==True].SUBJECT_ID.unique()))
print('Confirmation Number of patients on MechVent:', len(df[df.mv_documented==True].SUBJECT_ID.unique()))
print('New MechVent:', len(df[df.new_mv==True].SUBJECT_ID.unique()))
print('Confirmation New MechVent:', len(df[df.admit_date<df.mv_start_date].SUBJECT_ID.unique()))
print('')
MV_df=df[df.has_mv_start==True]
print('MV_df shape', MV_df.shape)
print('MV_df, new MechVent:', len(MV_df[MV_df.new_mv==True].SUBJECT_ID.unique()))

In [ ]:
df.columns

In [ ]:
def language(string): 
    asian=['MAND', 'CANT', 'CAMB', 'VIET', 'HIND', 'LAOT', 'KORE', 'THAI', 'JAPA']
    european=['GREE', 'RUSS', 'PORT', 'ITAL', 'POLI', 'FREN', 'TURK', 'GERM']
    if string in asian: return 'East_Southeast_Asian'
    elif string in european: return 'European_continental'
    elif string=='ENGL': return 'English'
    elif string=='SPAN': return 'Spanish'
    else: return 'Other/Unknown'

In [ ]:
df['language_summary']=df.LANGUAGE.map(lambda x: language(x))
df['English']=np.where(df.language_summary=='English', 1, 0)

In [ ]:
def ethnicity(string): 
    other=['UNABLE TO OBTAIN', 'UNKNOWN/NOT SPECIFIED', 'PATIENT DECLINED TO ANSWER', 'OTHER']
    native=['AMERICAN INDIAN/ALASKA NATIVE', 'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNIZED TRIBE']
    if 'ASIAN' in string: return 'Asian'
    elif 'WHITE' in string: return 'White'
    elif 'HISPANIC' in string: return 'Hispanic/Latino'
    elif 'BLACK' in string: return 'Black'
    elif string in native: return 'Native American or Hawaiian'
    elif string in other: return 'Unknown/Declined'
    else: return 'Multi/Other'

In [ ]:
df['ethnicity_summary']=df.ETHNICITY.map(lambda x: ethnicity(x))
df['White']=np.where(df.ethnicity_summary=='White', 1, 0)

In [ ]:
df['LOS']=df.LENGTH_OF_STAY.map(lambda x: int(''.join(re.findall(r'\d', str(x)[0:6]))) if 'days' in str(x) else 0)

In [ ]:
df['note_day']=df.day_of_note_from_admit.map(lambda x: int(''.join(re.findall(r'(\d+) days', x))))

In [ ]:
#Quick patch for AGE > 300 (should change in M001_merge)
df['AGE']=df.AGE.map(lambda x: np.nan if x>=300 else x)

In [ ]:
df['elixhauser_categories']=df.prior_comorbidities.map(lambda x: x.split(','))
pmh_dummies=pd.get_dummies(df.elixhauser_categories.apply(pd.Series).stack()).sum(level=0)
pmh_dummies['total_comorbidities']=pmh_dummies.sum(axis=1)
pmh_dummies

In [ ]:
df['num_elix_pmh']=pmh_dummies['total_comorbidities']

In [ ]:
df.head(5)

In [ ]:
avg_scores=df.groupby(['SUBJECT_ID', 'HADM_ID'], as_index=False)['tfidf_score', 'percent_neg_words', 'percent_pos_words'].mean()
avg_scores['avg_percent_neg_words']=avg_scores['percent_neg_words']
avg_scores['avg_percent_pos_words']=avg_scores['percent_pos_words']
avg_scores['avg_tfidf']=avg_scores['tfidf_score']
avg_scores.drop(['tfidf_score', 'percent_neg_words', 'percent_pos_words'], axis=1)

In [ ]:
len(df_by_day.HADM_ID.unique())

In [ ]:
df_by_day=df[['SUBJECT_ID', 'HADM_ID', 'CGID', 'AGE', 'GENDER', 'LOS', 'DC_TO_DEATH_HOSPICE', 'INSURANCE', 
       'language_summary', 'English', 'MARITAL_STATUS', 'ethnicity_summary', 'White', 'num_elix_pmh', 'OASIS_score', 'OASIS_prob', 
        'has_mv_start', 'has_HD_start', 'has_trach_start', 'has_pressors_start', 'note_day', 'tfidf_score', 
             'percent_neg_words', 'percent_pos_words']]

In [ ]:
#df_merge=pd.merge(df, avg_scores, on=['SUBJECT_ID', 'HADM_ID'], how='left')

In [ ]:
#df_by_day=df_merge[['SUBJECT_ID', 'HADM_ID', 'CGID', 'AGE', 'GENDER', 'LOS', 'DC_TO_DEATH_HOSPICE', 'INSURANCE', 
#       'language_summary', 'English', 'MARITAL_STATUS', 'ethnicity_summary', 'White', 'num_elix_pmh', 'OASIS_score', 'OASIS_prob', 
#        'has_mv_start', 'has_HD_start', 'has_trach_start', 'has_pressors_start', 'avg_tfidf', 'avg_percent_neg_words', 'avg_percent_pos_words']]

In [ ]:
df_by_day.to_csv('M006_all_pts_by_day.csv', index=False)

In [ ]:
summary_df=df_by_day.drop_duplicates(subset=['SUBJECT_ID', 'HADM_ID'], keep='last')

In [ ]:
summary_df.to_csv('M006_all_pts_avg.csv', index=False)

In [ ]:
summ_df_65=summary_df[summary_df.AGE>=65]
summ_df_65.to_csv('~/ccBox/Research_Project/NLP/Master_Files/M006_65_pts_avg.csv')

summ_df_mv=summary_df[summary_df.has_mv_start==1]
    summ_df_mv.to_csv('~/ccBox/Research_Project/NLP/Master_Files/M006_mv_pts_avg.csv')

summ_df_mv65=summary_df[(summary_df.has_mv_start==1)&(summary_df.AGE>=65)]
summ_df_mv65.to_csv('~/ccBox/Research_Project/NLP/Master_Files/M006_mv65_pts_avg.csv')